In [ ]:
import pandas as pd
import math
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.metrics import confusion_matrix
df=pd.read_excel('Synthetic Data_FA-ICC_28.07.2023 (003)_Statical.xlsx', sheet_name='Transactions') # datu ielade
df=df.iloc[:, : 15] # tikai pirmajas 15 kolonnas ir svariga info

In [ ]:
train, test, val = df[:240000], df[240000:270000], df[270000:] # nevajag randomizet, jo viss jau taa ir randoms

In [ ]:
train_np, test_np, val_np = train.to_numpy(), test.to_numpy(), val.to_numpy()
train_np.shape, test_np.shape, val_np.shape # parbaude

In [ ]:
x_train, y_train = train_np[:, :-1], train_np[:, -1]
x_test, y_test = test_np[:, :-1], test_np[:, -1]
x_val, y_val = val_np[:, :-1], val_np[:, -1]
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

In [ ]:
def neural_net_predictions(model, x):
    return (model.predict(x).flatten() > 0.5).astype(int) # nevajag speleties ar treshhold values, jo sis modelis ir vienkars, bet ja loti gribas....

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

# NN
model = Sequential()
model.add(InputLayer((x_train.shape[1],)))

# pirmais pasleptais slanis
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# otrais slanis
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# klasifikacija - ir fraud vai nav
model.add(Dense(1, activation='sigmoid'))

# kompilesana
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', 'Recall']) # recall, jo ir loti svarigi ka pareizi klasifice '1', kuru ir mazakums

# callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_recall', mode='max', save_best_only=True, verbose=1)
early_stopping = EarlyStopping(monitor='val_recall', mode='max', patience=5, verbose=1, restore_best_weights=True)

# train_model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,  # izvelets pec
    callbacks=[checkpoint, early_stopping],
    verbose=1
)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, neural_net_predictions(model, x_val), target_names=['Not Fraud', 'Fraud']))